In [0]:
# [1] GET INFORMATION OF MACHINE
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
from google.colab import auth
from googleapiclient.discovery import build
import io , requests, os
import sys
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
def get_files_from_parent(parent_id):
  page_token = None
  folder_array = dict()
  query = "'%s' in parents" % parent_id
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          folder_array.update({file.get('name'):file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array

In [0]:
def get_file_buffer(file_id, verbose=0):
  from googleapiclient.http import MediaIoBaseDownload
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      progress, done = downloader.next_chunk()
      if verbose:
        sys.stdout.flush()
        sys.stdout.write('\r')
        percentage_done = progress.resumable_progress * 100/progress.total_size
        sys.stdout.write("[%-100s] %d%%" % ('='*int(percentage_done), int(percentage_done)))
  downloaded.seek(0)
  return downloaded


In [0]:
def downloads_files_and_data(input_file_meta):
  SOURCE_FOLDER='/content'
  print('********** Download Logics Files **********')
  for file, id in input_file_meta.items():
    # If file is data -> need to go inside and download
    search_files(file, id, SOURCE_FOLDER)

In [0]:
def search_files(name, f_id, path):
  sub = get_files_from_parent(f_id)
  if len(sub) == 0:
    if not (os.path.exists(path)):  
      os.makedirs(path)
    try:
      downloaded = get_file_buffer(f_id, verbose=1)
      dest_data = os.path.join(path, name)
      print("processing %s data" % (path+"/"+name))
      with open(dest_data, "wb") as out:
        out.write(downloaded.read())
        print("Done data %s" % dest_data)
    except ValueError:
      print('SOME_THING_WENT_WRONG', ValueError)
  else:
    for n, i in sub.items():
      search_files(n, i, path+"/"+name )

In [0]:
def get_parent_folder(folder_name="Colab Notebooks"):
  page_token = None
  folder_array = []
  query = "name='%s' and mimeType='application/vnd.google-apps.folder'" % folder_name
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.append({"name" : file.get('name'), "id" : file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array


In [0]:
code_lab_folder = get_parent_folder()
sub_folder = get_files_from_parent(code_lab_folder[0]['id'])
print('code_lab' + str(code_lab_folder))
print('sub_folder ' + str(sub_folder))
a = list(sub_folder.keys())
print('Available folder: ')
for i in range(len(a)):
  print('---> '+str(i+1)+'. '+str(a[i]))
inp = int(input('Choose folder: '))
input_file_meta = get_files_from_parent(sub_folder[a[inp-1]])  
  
downloads_files_and_data(input_file_meta)

In [0]:
# sample running a python file from jupyter notebook
# !python softmax_tf.py